In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib
from time import sleep
import requests
import concurrent
from tqdm import tqdm

In [3]:
url_centro = 'https://www.zapimoveis.com.br/venda/casas/agr+sp+sao-paulo+regiao-do-centro/'
url_sul    = 'https://www.zapimoveis.com.br/venda/casas/agr+sp+sao-paulo+zona-sul/'
url_norte  = 'https://www.zapimoveis.com.br/venda/casas/agr+sp+sao-paulo+zona-norte/'
url_oeste  = 'https://www.zapimoveis.com.br/venda/casas/agr+sp+sao-paulo+zona-oeste/'
url_leste  = 'https://www.zapimoveis.com.br/venda/casas/agr+sp+sao-paulo+zona-leste/'

url_lista = [url_centro,url_sul,url_norte ,url_oeste,url_leste ]

In [4]:
headers = requests.utils.default_headers()

headers.update({"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'})

In [5]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in np.arange(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [6]:
# vamos pegar o numero de paginas da url
def numPag(url):
    lista_retorno = []
    for url_site in  url:
        r = requests.get(str(url_site), headers=headers)
        soup = BeautifulSoup(r.content, 'lxml')
        lista_retorno.append(int(soup.find('span',{'class':'pull-right num-of'}).text.replace('de ', '').replace('.', '')))
        sleep(2)
    # numero de paginas
    return lista_retorno

In [8]:
lista_num_pag = numPag(url_lista)

In [9]:
def soup_url(url, num_pag):
    lista = []
    if type(url) is list:
        ij=0
        for num in num_pag:
            for i in range(1, num+1):
                url_compl = '#{"pagina":"'+str(i)+'","formato":"Lista"}'
                lista.append(url[ij] + url_compl)
            ij=ij+1
    else:
        for i in range(1, num_pag+1):
            url_compl = '#{"pagina":"'+str(i)+'","formato":"Lista"}'
            lista.append(url + url_compl)
    return np.array(lista)

In [10]:
url_lista_todos = soup_url(url_lista, lista_num_pag)

In [11]:
len(url_lista_todos)

3264

In [12]:
def retornaSoup(url):
    r = requests.get(url, headers=headers)
#     soup = BeautifulSoup(r.content, 'lxml')
    sleep(3)
    return r.content

In [13]:
round(len(url_lista_todos)/90)

36

In [80]:
%%time
# retorna uma lista com a lista das soups de todas as paginas
lista_html = []
tamanho_batch = round((len(url_lista_todos)/100) * 8)
total = round(len(url_lista_todos)/tamanho_batch)

for lista in tqdm(batch(url_lista_todos, tamanho_batch),total=total):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        result = executor.map(retornaSoup, lista)
        lista_html.append(list(result))


  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
lista_html_aux = [i for j in lista_html for i in j]

In [16]:
def retornaBSoup(html):
    return BeautifulSoup(html, 'lxml')

In [ ]:
np.savez_compressed('save', lista_html_aux)

In [70]:
lista_soup = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    lista_soup.append(executor.map(retornaBSoup, lista_html_aux[:10]))

In [73]:
x = [j for i in lista_soup for j in i]

In [ ]:
%%time
# lista com todos os articles da lista criada anteriormente
x = [article for lista_htmls in lista_soup for html in lista_htmls for article in html.find_all('article')]

In [ ]:
%%time
NoneType = type(None)
href         = [i.find('a')['href'] if type(i.find('a')) is not NoneType else 'sem url' for i in x]
endereco     = [i.find('span', {'itemprop':'streetAddress'}).text if type(i.find('span', {'itemprop':'streetAddress'})) is not NoneType else 'sem emdereco' for i in x]
preco        = [i.find('div',{'class':'preco'}).find('strong').text if type(i.find('div',{'class':'preco'}).find('strong')) is not NoneType else 0 for i in x]
info_quartos = [i.find('li',{'class':'icone-quartos'}).text if type(i.find('li',{'class':'icone-quartos'})) is not NoneType else 0 for i in x]
info_area    = [i.find('li',{'class':'icone-area'}).text if type(i.find('li',{'class':'icone-area'})) is not NoneType else 0 for i in x]
info_suites  = [i.find('li',{'class':'icone-suites'}).text if type(i.find('li',{'class':'icone-suites'})) is not NoneType else 0 for i in x]
info_area    = [i.find('li',{'class':'icone-area'}).text if type(i.find('li',{'class':'icone-area'})) is not NoneType else 0 for i in x]

In [ ]:
x2 = [list(e) for e in zip(href,endereco,preco,info_quartos,info_area,info_suites,info_area)]

In [ ]:
df = pd.DataFrame(x2)

In [5]:
x = [10,9,8,7,6,5,4,3,2,1]